In [ ]:
import tensorflow as tf
import zipfile
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from keras.applications.resnet50 import ResNet50 , preprocess_input
from numpy.linalg import norm
import pandas as pd

In [ ]:
!kaggle datasets download -d hiteshsuthar101/myntra-fashion-product-dataset

100% 2.90G/2.90G [00:30<00:00, 129MB/s]
100% 2.90G/2.90G [00:30<00:00, 101MB/s]


In [ ]:
zipref = zipfile.ZipFile('myntra-fashion-product-dataset.zip')
zipref.extractall()
zipref.close()

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import numpy as np
import pandas as pd
import ast
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


import ast
df=pd.read_csv("/content/Fashion Dataset.csv")
df.head()
df_c=df.dropna()
df_c=df_c[["name","colour","brand","description","p_attributes","img","price"]]
def convert(ob):
    l=[]
    for i,j in ob.items():
        if i==["Number of Pockets"]:
            l.append(v[i])
    return l
def convertd(df):
    v=ast.literal_eval(df)


    l=[]
    for i in v.keys():
        if i in ["Number of Pockets","Stitch","Wash Care"]:
            l.append(v[i])

    return l

df_c["p_attributes"]=df_c["p_attributes"].apply(convertd)
def list_conv(text):
    l=[]
    for i in text.split():
        l.append(i)
    return l
df_c["brand"]=df_c["brand"].apply(list_conv)
df_c["brand"]=df_c["brand"].apply(lambda x:[i.replace(" ","") for i in x])
df_c["colour"]=df_c["colour"].apply(list_conv)
df_c["colour"]=df_c["colour"].apply(lambda x:[i.replace(" ","") for i in x])
df_c["p_attributes"]=df_c["p_attributes"].apply(lambda x:[i.replace(" ","") for i in x])
df_c["description"]=df_c["description"].apply(list_conv)
df_c
df_c["Tag"]=df_c["colour"]+df_c["brand"]+df_c["description"]+df_c["p_attributes"]
def st(text):
    l=[]
    for i in text.split():
        l.append(ps.stem(i))
    return " ".join(l)
import nltk
from nltk.stem.porter import PorterStemmer

ps=PorterStemmer()
df_c["Tag"]=df_c["Tag"].apply(lambda x: " ".join(x))
df_c["Tag"]=df_c["Tag"].apply(st)
df_c=df_c[["name","Tag","img"]]
df_c
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=6000,stop_words="english")
vectors=cv.fit_transform(df_c["Tag"]).toarray()
from sklearn.metrics.pairwise import cosine_similarity
# Calculate cosine distances between vectors
distances = cosine_distances(vectors)
def similar_images(movie):
    index = df_c[df_c["name"] == movie].index[0]
    distance = distances[index]
    # Sort distances in ascending order to find similar images
    list_v = sorted(list(enumerate(distance)), key=lambda x: x[1])[:6]  # Change [1:6] to [0:6] to include the input image

    for i in list_v:
        c = i[0]
        print(c, df_c.iloc[c]["name"], df_c.iloc[c]["img"], 'Rs.' + str(df.iloc[c]["price"]))

similar_images("Khushal K Women White Ethnic Motifs Printed Gotta Patti Kurta with Palazzos & With Dupatta")



472 Anouk X Earthful Women White & Pink Floral Printed A-Line Kurta http://assets.myntassets.com/assets/images/16272186/2022/4/12/3fbb2568-3dd1-472a-9971-d9c1c5d800931649758997074-Anouk-Women-White--Pink-Floral-Printed-Pastels-Kurta-2581649-1.jpg Rs.4799.0
480 Anouk Women White & Blue Floral Printed Kurta http://assets.myntassets.com/assets/images/16190856/2022/5/6/60bc83ec-edd9-4d28-ae8d-1efc66afcbfe1651820015650-Anouk-Women-Kurtas-61651820015285-5.jpg Rs.3399.0
516 HERE&NOW Women Blue & White Striped Embroidered Mirror Work A-Line Kurta http://assets.myntassets.com/assets/images/17279494/2022/5/10/b9fd2236-0ea1-4990-91c9-d3ea95768c5c1652164055648-HERENOW-Women-Kurtas-7801652164055110-1.jpg Rs.1699.0
288 Anouk Women Black & White Bandhani Printed Halter Neck Kurta http://assets.myntassets.com/assets/images/18533904/2022/6/17/3587d447-0b2a-4326-b5e2-fb439066331b1655443585896-Anouk-Women-Kurtas-2981655443585291-1.jpg Rs.2999.0
648 Anouk Women Pink Pure Cotton Solid Kurta http://assets.m

In [ ]:
import pandas as pd

# Load the fashion dataset CSV file
df = pd.read_csv("/content/Fashion Dataset.csv")

# Define file paths for different categories
category_files = {
    "outfit": "outfit_items.csv",
    "top": "top_items.csv",
    "pants": "pants_items.csv",
}

# Function to determine category based on keywords in the name
def get_category(name):
    name = name.lower()  # Convert to lowercase for case-insensitive matching
    if any(keyword in name for keyword in ["dupatta",'lehenga','saree',"kurti", "kurta", "palazzos",'tunic','jumpsuit','dress','co-ord','playsuit']):
        return "outfit"
    elif any(keyword in name for keyword in ["top","jacket","blazer","shrug","pullover","sweatshirt","sweater","cardigan"]):
        return "top"
    elif any(keyword in name for keyword in ["pants","leggings", "jeans","trousers",'joggers',"culottes",]):
        return "pants"
    else:
        return None

# Iterate through each row of the "name" column and write to respective category files
for index, row in df.iterrows():
    name = row["name"]
    if isinstance(name, str):
        category = get_category(name)
        if category:
            with open(category_files[category], "a") as f:
                f.write(','.join(map(str, row)) + '\n')
        else:
            print(f"No category found for item '{name}'. Skipping...")
    else:
        print(f"Invalid name for item at index {index}. Skipping...")

print("Rows appended to respective category files.")


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the top and pant items data
top_items_df = pd.read_csv('/content/top_items_desc.csv')
pant_items_df = pd.read_csv('/content/pants_items_desc.csv')

# Preprocess the data
top_items_descriptions = top_items_df['description'].fillna('')
pant_items_descriptions = pant_items_df['description'].fillna('')

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Fit TF-IDF vectorizer on top items descriptions
tfidf_vectorizer.fit(top_items_descriptions)

# Transform top and pant items descriptions to TF-IDF vectors
top_items_tfidf = tfidf_vectorizer.transform(top_items_descriptions)
pant_items_tfidf = tfidf_vectorizer.transform(pant_items_descriptions)

# Compute cosine similarity between top and pant items
cosine_similarities = cosine_similarity(top_items_tfidf, pant_items_tfidf)


In [ ]:
def recommend_pant_for_top(top_description):
    top_tfidf = tfidf_vectorizer.transform([top_description])
    similarities = cosine_similarity(top_tfidf, pant_items_tfidf)

    # Take the negative of cosine similarity to emphasize complementarity
    complementarity_scores = -similarities

    best_pant_indices = complementarity_scores.argsort(axis=None)[-5:][::-1]

    return pant_items_df.iloc[best_pant_indices]

def recommend_top_for_pant(pant_description):

    pant_tfidf = tfidf_vectorizer.transform([pant_description])

    similarities = cosine_similarity(top_items_tfidf, pant_tfidf)

    # Take the negative of cosine similarity to emphasize complementarity
    complementarity_scores = -similarities

    best_top_indices = complementarity_scores.argsort(axis=None)[-5:][::-1]

    return top_items_df.iloc[best_top_indices]


In [ ]:
import pandas as pd

# Set display options
pd.set_option('display.max_colwidth', None)
# Get user input for a top description
user_top_description = input("Enter the description of the top: ")

# Recommend complementary pant items for the entered top description
recommended_pants = recommend_pant_for_top(user_top_description)
print("\nRecommended Pant Items:")
print(recommended_pants[['name', 'img', 'price']])

# Get user input for a pant description
user_pant_description = input("\nEnter the description of the pant: ")

# Recommend complementary top items for the entered pant description
recommended_tops = recommend_top_for_pant(user_pant_description)
print("\nRecommended Top Items:")
print(recommended_tops[['name', 'img', 'price']])

Enter the description of the top: white shirt

Recommended Pant Items:
                                                                                     name  \
978                             BoStreet Women Black Loose Fit High-Rise Hot Pants Shorts   
676  SHOWOFF Women Blue Boyfriend Fit High-Rise Low Distress Light Fade Stretchable Jeans   
648                                Campus Sutra Women Navy Blue Slim Fit Heavy Fade Jeans   
649                            URBANIC Women Blue Straight Fit Stylized Stretchable Jeans   
650                                      GOLDSTROMS Women Blue Slim Fit Stretchable Jeans   

                                                                                                                                                                                  img  \
978                                                        http://assets.myntassets.com/assets/images/18458288/2022/5/27/8b64ea90-ab79-4a6e-ae86-cd393a830c941653660239512Shorts1.jpg   
